<p style="font-weight:bold;"> <span style="font-size: 36px"> Interact With Import Scope Calculation</span> </p>

<br><br>
After successfully loading the parsed data in the dataSource, the import scopes for a specific partition can be re-evaluated in this notebook. 
The import scopes can then be displayed, examined and used for further calculations.

# Basic Initialization

In [0]:
#!import "../../Constants/CalculationEngine"

In [0]:
//#!eval-notebook "../../Initialization/InitSystemorphBaseToMemory"
#!eval-notebook "../../Initialization/InitSystemorphToMemory"
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.InitializeFrom(DataSource);

# Import of parsed data

In [0]:
Debug.Enable = true;

In [0]:
await Import.FromFile("../../Files/TransactionalData/NominalCashflows_CH_2021_3_Projection.csv")
            .WithFormat(ImportFormats.Cashflow)
            .WithTarget(Workspace)
            .WithActivityLog()
            .ExecuteAsync()

# Args

## Primary Args

In [0]:
var reportingNode = "CH";
var year = 2021;
var month = 3;
var importFormat = ImportFormats.Cashflow;
var scenario = (string)null;

ImportArgs args = new ImportArgs(reportingNode, year, month, default(Periodicity), scenario, importFormat);
args

## Secondary Args

In [0]:
var allArgs = await GetAllArgsAsync(args, DataSource, ImportFormats.Cashflow);
allArgs

# Construct ImportStorage

In [0]:
var storage = new ImportStorage(args, DataSource, Workspace);
await storage.InitializeAsync();

# Compute Ifrs Variables

In [0]:
var universe = Scopes.ForStorage(storage).ToScope<IModel>();

In [0]:
var allIvs = Scopes.ForIdentities(storage.DataNodesByImportScope[ImportScope.Primary], storage).ToScopes<ComputeAllScopes>().SelectMany(x => x.CalculatedIfrsVariables).ToArray();

In [0]:
Workspace.Query<RawVariable>().Where(x => x.DataNode =="3BPAA.0")

In [0]:
storage.GetIfrsVariables("EY59G.1").Max(ifrsVariable => ifrsVariable.Values.Length)

In [0]:
storage.DataNodesByImportScope

In [0]:
Scopes.ForIdentities(new []{"3BPAA.0"}, storage).ToScopes<ComputeIfrsVarsCashflowsWithProjection>().SelectMany(x => x.CalculatedIfrsVariables)

In [0]:
storage.GetIfrsVariables("EY52PAA.1").Any()//.Max(ifrsVariable => ifrsVariable.Values.Length)

In [0]:
Enumerable.Range(0,1)

## Select Identity and projection to debug

In [0]:
var identities = universe.GetScopes<GetIdentities>(storage.DataNodesByImportScope[ImportScope.Primary]).SelectMany(s => s.Identities);
var idByDn = identities.ToDictionaryGrouped(x => x.DataNode, x => x.ToArray());

In [0]:
identities.Where(x => x.AocType =="WO")

In [0]:
int projection = 1;

In [0]:
var ivs = Scopes.ForIdentities(identities.Select(id => id with {ProjectionPeriod = projection}), storage).ToScopes<PvToIfrsVariable>().SelectMany(x => x.PvLocked.Concat(x.PvCurrent)).ToArray();

In [0]:
var ivs = Scopes.ForIdentities(identities.Select(id => id with {ProjectionPeriod = projection}), storage).ToScopes<NominalToIfrsVariable>().SelectMany(x => x.CumulatedNominal).ToArray();

In [0]:
var ivs = Scopes.ForIdentities(identities.Select(id => id with {ProjectionPeriod = projection}), storage).ToScopes<RaToIfrsVariable>().SelectMany(x => x.RaCurrent.Concat(x.RaLocked)).ToArray();

In [0]:
var ivs = Scopes.ForIdentities(identities.Select(id => id with {ProjectionPeriod = projection}), storage).ToScopes<ActualToIfrsVariable>().SelectMany(x => x.Actual.Concat(x.AdvanceActual).Concat(x.OverdueActual)).ToArray();

In [0]:
var ivs = Scopes.ForIdentities(identities.Select(id => id with {ProjectionPeriod = projection}), storage).ToScopes<DeferrableToIfrsVariable>().SelectMany(x => x.Deferrable).ToArray();

In [0]:
var ivs = Scopes.ForIdentities(identities.Select(id => id with {ProjectionPeriod = projection}), storage).ToScopes<EaForPremiumToIfrsVariable>().SelectMany(x => x.BeEAForPremium.Concat(x.ActEAForPremium)).ToArray();

In [0]:
var ivs = Scopes.ForIdentities(identities.Select(id => id with {ProjectionPeriod = projection}), storage).ToScopes<TmToIfrsVariable>().SelectMany(x => x.AmortizationFactor.Concat(x.Csms).Concat(x.Loss)).ToArray();

In [0]:
ivs